# Yahoo Finance から Currentの金額を取得する。

In [5]:
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError
import pandas as pd
import psycopg2

# 目的変数を作成する
def stockCurValue(code,S_year,S_day):
    company_code = str(code) + '.T'
    my_share = share.Share(company_code)
    symbol_data = None

    try:
        symbol_data = my_share.get_historical(share.PERIOD_TYPE_YEAR,S_year,share.FREQUENCY_TYPE_DAY,S_day)
    except YahooFinanceError as e:
        print(e.message)
        sys.exit(1)

    print(symbol_data)
    # 株価をデータフレームに入れている
    df_base = pd.DataFrame(symbol_data)
    df_base = pd.DataFrame(symbol_data.values(), index=symbol_data.keys()).T
    df_base.timestamp = pd.to_datetime(df_base.timestamp, unit='ms')
    df_base.index = pd.DatetimeIndex(df_base.timestamp, name='timestamp').tz_localize('UTC').tz_convert('Asia/Tokyo')
    #df_base = df_base.drop(['timestamp', 'open', 'high', 'low', 'volume'], axis=1)
    
    #df_base = df_base.rename(columns={'close':company_code + '対象'})
    #df_base = df_base[:-1] #一番最後の行を削除
    df_base = df_base.reset_index(drop=True)
    
    
    return company_code, df_base

# Connect to your postgres DB
conn = psycopg2.connect("host=localhost dbname=Life user=postgres password=YTK@pg")

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a query
cur.execute("SELECT * from finance.m_securities")

# Retrieve query results
records = cur.fetchall()

records

code = 3323 #ターゲット
S_year = 0.2 #取得年数
S_day = 1 #取得単位

result = stockCurValue(code,S_year,S_day)
print(str(result[0]), result[1].shape)

# データフレームへもどす
df_base = result[1]

df_base.head()

{'timestamp': [1706140800000, 1706227200000, 1706486400000, 1706572800000, 1706659200000, 1706745600000, 1706832000000, 1707091200000, 1707177600000, 1707264000000, 1707350400000, 1707436800000, 1707782400000, 1707868800000, 1707955200000, 1708041600000, 1708300800000, 1708387200000, 1708473600000, 1708560000000, 1708905600000, 1708992000000, 1709078400000, 1709164800000, 1709251200000, 1709510400000, 1709596800000, 1709683200000, 1709769600000, 1709856000000, 1710115200000, 1710201600000, 1710288000000, 1710374400000, 1710460800000, 1710720000000, 1710806400000, 1710979200000, 1711065600000, 1711324800000, 1711411200000, 1711497600000, 1711584000000, 1711670400000, 1711929600000, 1712016000000, 1712102400000, 1712188800000, 1712275200000], 'open': [93.0, 94.0, 95.0, 97.0, 93.0, 92.0, 93.0, 93.0, 95.0, 94.0, 93.0, 90.0, 90.0, 90.0, 92.0, 88.0, 88.0, 89.0, 91.0, 91.0, 90.0, 92.0, 91.0, 92.0, 94.0, 91.0, 90.0, 90.0, 93.0, 91.0, 90.0, 88.0, 90.0, 87.0, 88.0, 88.0, 88.0, 88.0, 90.0, 91.0, 

,timestamp,open,high,low,close,volume
0,2024-01-25,93.0,94.0,92.0,93.0,404000.0
1,2024-01-26,94.0,96.0,94.0,94.0,738900.0
2,2024-01-29,95.0,99.0,95.0,96.0,1019700.0
3,2024-01-30,97.0,98.0,96.0,97.0,280500.0
4,2024-01-31,93.0,93.0,93.0,93.0,18500.0
